# Convert file into COG or ZARR


In [1]:
## Imports

import os
import glob
from pathlib import Path
from datetime import datetime
rom tools.emit_tools_new import emit_xarray, band_mask

import xarray as xr
import rioxarray as rxr
import zarr
import subprocess

## EMIT

In [2]:
rfl_paths = glob.glob('/mnt/frontiersi-hyperspectral/EMITL2ARFL.001/EMIT_L2A_RFL_001_*/EMIT_L2A_RFL_*.nc')

In [3]:
rfl_paths

['/mnt/frontiersi-hyperspectral/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230316T045211_2307503_006/EMIT_L2A_RFL_001_20230316T045211_2307503_006.nc',
 '/mnt/frontiersi-hyperspectral/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230919T025123_2326201_005/EMIT_L2A_RFL_001_20230919T025123_2326201_005.nc',
 '/mnt/frontiersi-hyperspectral/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240325T002836_2408415_019/EMIT_L2A_RFL_001_20240325T002836_2408415_019.nc',
 '/mnt/frontiersi-hyperspectral/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240901T013648_2424501_059/EMIT_L2A_RFL_001_20240901T013648_2424501_059.nc',
 '/mnt/frontiersi-hyperspectral/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240923T002255_2426615_009/EMIT_L2A_RFL_001_20240923T002255_2426615_009.nc',
 '/mnt/frontiersi-hyperspectral/EMITL2ARFL.001/EMIT_L2A_RFL_001_20241029T023534_2430302_057/EMIT_L2A_RFL_001_20241029T023534_2430302_057.nc',
 '/mnt/frontiersi-hyperspectral/EMITL2ARFL.001/EMIT_L2A_RFL_001_20250305T004702_2506401_039/EMIT_L2A_RFL_001_20250305T004702_2506401_039.nc']

In [4]:
# COG

for rfl_path in rfl_paths:
    tif_path = rfl_path.replace('.nc', '_masked.tif')
    if os.path.exists(tif_path): continue    
        
    if rfl_path.endswith('005.nc'): continue
        
    print(rfl_path)
    # find mask
    mask_path = Path(rfl_path).parent / Path(rfl_path).name.replace('_RFL_', '_MASK_')
    ds_mask = band_mask(mask_path)
    # load reflectance masked 
    ds_masked = emit_xarray(rfl_path, unpacked_bmask= ds_mask)
        
    wave_int = ds_masked.coords["bands"].values.astype(int).astype(str)
    ds_masked = ds_masked.reflectance.to_dataset(dim='bands').drop_dims('bands').rename(dict(zip(ds_masked.coords["bands"].values, wave_int)))
    ds_masked.rio.to_raster(
            tif_path,
            compress="DEFLATE",           # Compression for COG
            predictor=2,              # Improves compression efficiency
            tiled=True,               # Enable tiling
            blockxsize=512,           # Tile size
            blockysize=512,
            interleave="band",        # Organizes bands separately
            bigtiff="YES",
    )
    del ds_mask, ds_masked

/mnt/frontiersi-hyperspectral/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240325T002836_2408415_019/EMIT_L2A_RFL_001_20240325T002836_2408415_019.nc


In [6]:
# ZARR

for rfl_path in rfl_paths:
    zarr_path = rfl_path.replace('.nc', '_masked.zarr')
    if os.path.exists(zarr_path): continue

    if rfl_path.endswith('005.nc'): continue
    
    zarr_local = Path(zarr_path).name
    print(rfl_path)
    
    # find mask
    mask_path = Path(rfl_path).parent / Path(rfl_path).name.replace('_RFL_', '_MASK_')
    ds_mask = band_mask(mask_path)
    # load reflectance masked 
    ds_masked = emit_xarray(rfl_path, unpacked_bmask= ds_mask)

    ds_masked.to_zarr(zarr_local, mode="w")
    cmd = f"mv {zarr_local} {Path(rfl_path).parent}/"
    subprocess.run(cmd.split(), check=True)

/mnt/frontiersi-hyperspectral/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240901T013648_2424501_059/EMIT_L2A_RFL_001_20240901T013648_2424501_059.nc


/home/ec2-user/miniconda3/envs/emit/lib/python3.13/site-packages/zarr/api/asynchronous.py:203: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


## ENMAP

ENMAP seems already in COG

In [6]:
enmap_paths = glob.glob('/mnt/frontiersi-hyperspectral/ENMAP/ENMAP01-*.TIF')

In [7]:
enmap_paths

['/mnt/frontiersi-hyperspectral/ENMAP/ENMAP01-____L2A-DT0000063368_20240229T005105Z_015_V010401_20240306T141210Z-SPECTRAL_IMAGE_COG.TIF',
 '/mnt/frontiersi-hyperspectral/ENMAP/ENMAP01-____L2A-DT0000064649_20240308T005824Z_002_V010401_20240309T171044Z-SPECTRAL_IMAGE_COG.TIF',
 '/mnt/frontiersi-hyperspectral/ENMAP/ENMAP01-____L2A-DT0000064649_20240308T005828Z_003_V010401_20240309T171343Z-SPECTRAL_IMAGE_COG.TIF']

In [10]:
ds = rxr.open_rasterio(enmap_paths[0], chunks={})

In [11]:
ds

<xarray.DataArray (band: 224, y: 1184, x: 1232)> Size: 653MB
dask.array<open_rasterio-794706015695f6f3dbca7fcc3935fb6e<this-array>, shape=(224, 1184, 1232), dtype=int16, chunksize=(224, 1184, 1232), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 2kB 1 2 3 4 5 6 7 ... 218 219 220 221 222 223 224
  * x            (x) float64 10kB 4.655e+05 4.655e+05 ... 5.024e+05 5.024e+05
  * y            (y) float64 9kB 5.819e+06 5.819e+06 ... 5.783e+06 5.783e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     -32768
    scale_factor:   1.0
    add_offset:     0.0